In [5]:
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------------
# Load files
# ------------------------------------------------------------------
# FULL      = Path("actor_level_data/ICB_ATOP_actor_level_20250702.csv")
ANALYSIS  = Path("actor_level_data/ICB_ATOP_actor_level_20250702_filtered.csv")

# full_df    = pd.read_csv(FULL)
analysis_df = pd.read_csv(ANALYSIS)

# Helper: parse yyyy-mm-dd safely
to_dt = lambda s: pd.to_datetime(s, errors="coerce")

# ══════════════════════════════════════════════════════════════════
# (A)  FILE-WIDE INTEGRITY TESTS
# ══════════════════════════════════════════════════════════════════


""" 
# 3.  Actor_TrigDate ≤ Actor_TermDate
bad = analysis_df[to_dt(analysis_df["Actor_TrigDate"]) >
                  to_dt(analysis_df["Actor_TermDate"])]
assert bad.empty, "Found actor rows with TrigDate > TermDate" """

# 4.  Actor dates lie inside treaty window when Actor_Is_Member == 1
mask = analysis_df["Actor_Is_Member"] == 1
bad = analysis_df[mask & (
        (to_dt(analysis_df["Actor_TermDate"]) <
         to_dt(analysis_df["Actor_Member_Start"])) |
        (to_dt(analysis_df["Actor_TrigDate"])  >
         to_dt(analysis_df["Actor_Member_End"]))
)]
assert bad.empty, "Actor_Is_Member rows violate membership window"

# 5.  No duplicate (Crisis_ID, Actor_ID, Alliance_ID)
dupes = analysis_df.duplicated(["Crisis_ID","Actor_ID","Alliance_ID"]).sum()
assert dupes == 0, f"Duplicates in analysis file: {dupes}"

print("✓ Global integrity tests passed")

# ══════════════════════════════════════════════════════════════════
# (B)  SPOT-CHECKS ON HISTORICAL CASES
# ══════════════════════════════════════════════════════════════════

def has_pair(df, crisis, actor, alliance, expect):
    """Return True if dataset contains expected match"""
    row = df[(df.Crisis_ID==crisis) &
             (df.Actor_ID==actor) &
             (df.Alliance_ID==alliance)]
    return (not row.empty) and (row.iloc[0]["Both_Conditions"]==expect)

# 6.  1956 Hungarian Uprising × NATO (actor=Hungary 310)  → expect 0
assert has_pair(full_df, 155, 310, 3180, 0), \
    "Hungary-NATO 1956 incorrectly marked true"

# 7.  1956 Hungarian Uprising × Warsaw Pact (actor=Hungary) → expect 1
assert has_pair(analysis_df, 155, 310, 3185, 1), \
    "Hungary-WarsawPact 1956 missing or false"

# 8.  1938 Munich Crisis × Franco-Czech Treaty (actor=Czechoslovakia 315) → 1
assert has_pair(analysis_df, 64, 315, 2120, 1), \
    "Franco-Czech Treaty 1938 not flagged true"

# 9.  2003 Iraq War × NATO (actor=USA 2) → expect 1
assert has_pair(analysis_df, 442, 2, 3180, 1), \
    "USA-NATO 2003 Iraq War missing/false"

print("✓ All spot-checks passed")


AssertionError: Duplicates in analysis file: 49

In [6]:
# ── Inspect duplicate (Crisis_ID, Actor_ID, Alliance_ID) rows ──────────────
key = ["Crisis_ID", "Actor_ID", "Alliance_ID"]

dup_mask = analysis_df.duplicated(key, keep=False)      # flag every copy
dupes_df = analysis_df[dup_mask].sort_values(key)       # 49 rows in your case

print(f"\n⚠️  Duplicate triplets found: {dupes_df.shape[0]} rows "
      f"across {dupes_df[key].drop_duplicates().shape[0]} unique keys\n")

# show how many copies per key
count_tbl = (dupes_df.groupby(key)
                       .size()
                       .reset_index(name="Copies")
                       .sort_values("Copies", ascending=False))
print("Copies per (Crisis, Actor, Alliance):")
print(count_tbl.head(10).to_string(index=False))        # top 10 offenders

# full details for spreadsheet review
dup_file = Path("actor_level_data/duplicate_triplets.csv")
dupes_df.to_csv(dup_file, index=False)
print(f"\n✓ All duplicate rows written to {dup_file}")



⚠️  Duplicate triplets found: 94 rows across 45 unique keys

Copies per (Crisis, Actor, Alliance):
 Crisis_ID  Actor_ID  Alliance_ID  Copies
        80       360         2085       3
        80       360         2215       3
        80       360         2315       3
        80       360         2330       3
        21       315         2050       2
       242       770         3222       2
       216       750         3225       2
       232       651         3015       2
       232       651         3205       2
       232       651         3350       2

✓ All duplicate rows written to actor_level_data\duplicate_triplets.csv


In [7]:
import pandas as pd
from pathlib import Path

# Already have `analysis_df` and the duplicate mask `dup_mask`
dup_df = analysis_df[dup_mask].copy()

# ------------------------------------------------------------------
# choose crises to inspect ─ default = first 3 crises with dups
# ------------------------------------------------------------------
crises_to_check = (dup_df["Crisis_ID"]
                   .value_counts()
                   .head(3)               # change .head(…) or set manually
                   .index.tolist())
print(f"\nInspecting crises: {crises_to_check}")

for cid in crises_to_check:
    slice_df = dup_df[dup_df["Crisis_ID"] == cid].sort_values(
                   ["Actor_ID", "Alliance_ID"])
    print("\n" + "="*70)
    print(f"CRISIS {cid}: {slice_df.iloc[0]['Crisis_Name']}")
    print(f"Duplicate rows: {len(slice_df)}")

    # group by (Actor, Alliance) to spot field-level differences
    for (actor, aid), group in slice_df.groupby(["Actor_ID","Alliance_ID"]):
        if len(group) > 1:
            # find columns that vary within the group
            var_cols = [col for col in group.columns
                        if len(group[col].drop_duplicates()) > 1]
            print(f"\n— Actor {actor}, Alliance {aid} appears {len(group)}×; "
                  f"varying columns: {var_cols}")
            print(group[var_cols + ["Actor_TrigDate","Actor_TermDate"]]
                  .to_string(index=False))



Inspecting crises: [83, 232, 80]

CRISIS 83: BALKAN INVASIONS
Duplicate rows: 18

— Actor 200, Alliance 1400 appears 2×; varying columns: ['Actor_TrigDate', 'Actor_TermDate']
Actor_TrigDate Actor_TermDate Actor_TrigDate Actor_TermDate
    1940-10-28     1940-11-22     1940-10-28     1940-11-22
    1941-04-06     1941-06-01     1941-04-06     1941-06-01

— Actor 200, Alliance 2270 appears 2×; varying columns: ['Actor_TrigDate', 'Actor_TermDate']
Actor_TrigDate Actor_TermDate Actor_TrigDate Actor_TermDate
    1940-10-28     1940-11-22     1940-10-28     1940-11-22
    1941-04-06     1941-06-01     1941-04-06     1941-06-01

— Actor 200, Alliance 2385 appears 2×; varying columns: ['Actor_TrigDate', 'Actor_TermDate']
Actor_TrigDate Actor_TermDate Actor_TrigDate Actor_TermDate
    1940-10-28     1940-11-22     1940-10-28     1940-11-22
    1941-04-06     1941-06-01     1941-04-06     1941-06-01

— Actor 200, Alliance 2490 appears 2×; varying columns: ['Actor_TrigDate', 'Actor_TermDate']
Ac

In [8]:
# ── Add Crisis_Phase to actor-level datasets ──────────────────────
import pandas as pd
from pathlib import Path

FULL_CSV = Path("actor_level_data/ICB_ATOP_actor_level_20250702.csv")
FILT_CSV = Path("actor_level_data/ICB_ATOP_actor_level_20250702_filtered.csv")

def add_crisis_phase(df: pd.DataFrame) -> pd.DataFrame:
    # Parse dates so phases are ordered chronologically
    trig = pd.to_datetime(df["Actor_TrigDate"],  errors="coerce")
    term = pd.to_datetime(df["Actor_TermDate"],  errors="coerce")

    # Order rows and assign phase numbers per Crisis–Actor pair
    df = (df.assign(_TrigDt=trig, _TermDt=term)
            .sort_values(["Crisis_ID", "Actor_ID", "_TrigDt", "_TermDt"])
            .assign(Crisis_Phase=lambda x:
                    x.groupby(["Crisis_ID", "Actor_ID"]).cumcount() + 1)
            .drop(columns=["_TrigDt", "_TermDt"]))
    return df

# 1) Full actor-level file
actor_full = pd.read_csv(FULL_CSV)
actor_full = add_crisis_phase(actor_full)
actor_full.to_csv(FULL_CSV, index=False)
print(f"✓ updated {FULL_CSV} with Crisis_Phase")

# 2) Filtered analysis subset (Both_Conditions == 1)
actor_filt = actor_full[actor_full["Both_Conditions"] == 1].copy()
actor_filt.to_csv(FILT_CSV, index=False)
print(f"✓ regenerated {FILT_CSV} (rows kept: {len(actor_filt):,})")


✓ updated actor_level_data\ICB_ATOP_actor_level_20250702.csv with Crisis_Phase
✓ regenerated actor_level_data\ICB_ATOP_actor_level_20250702_filtered.csv (rows kept: 5,102)


In [9]:
import pandas as pd
from pathlib import Path

FULL = Path("actor_level_data/ICB_ATOP_actor_level_20250702.csv")
FILT = Path("actor_level_data/ICB_ATOP_actor_level_20250702_filtered.csv")

def duplicate_report(path: Path) -> None:
    df = pd.read_csv(path)
    key = ["Crisis_ID", "Actor_ID", "Alliance_ID", "Crisis_Phase"]

    dup_total = df.duplicated(key, keep=False).sum()
    uniq_keys = df[df.duplicated(key, keep=False)][key].drop_duplicates().shape[0]

    print(f"\nFile: {path.name}")
    print(f"  Duplicate rows: {dup_total}")
    print(f"  Affected unique keys: {uniq_keys}")

    if dup_total:
        # show first few offending keys
        bad = (df[df.duplicated(key, keep=False)]
               .groupby(key).size()
               .reset_index(name="Copies")
               .sort_values("Copies", ascending=False)
               .head(10))
        print("\n  Top duplicate keys (showing up to 10):")
        print(bad.to_string(index=False))
    else:
        print("  ✓ No duplicates under the new key")

duplicate_report(FULL)
duplicate_report(FILT)



File: ICB_ATOP_actor_level_20250702.csv
  Duplicate rows: 0
  Affected unique keys: 0
  ✓ No duplicates under the new key

File: ICB_ATOP_actor_level_20250702_filtered.csv
  Duplicate rows: 0
  Affected unique keys: 0
  ✓ No duplicates under the new key


In [4]:
# ── Inspect rows where Actor_TrigDate > Actor_TermDate ──────────
bad_mask = to_dt(analysis_df["Actor_TrigDate"]) > to_dt(analysis_df["Actor_TermDate"])
bad = analysis_df[bad_mask].copy()

print(f"\n⚠️  {len(bad)} rows have Actor_TrigDate later than Actor_TermDate\n")
if not bad.empty:
    show_cols = list(bad.columns)      # ← include ALL columns
    with pd.option_context("display.max_columns", None,
                           "display.width", 200):
        print(bad[show_cols].to_string(index=False))

    # save full details for spreadsheet review
    bad_file = Path("actor_level_data/inconsistent_actor_dates.csv")
    bad.to_csv(bad_file, index=False)
    print(f"\n✓ Detailed list written to {bad_file}")



⚠️  5 rows have Actor_TrigDate later than Actor_TermDate

 Crisis_ID      Crisis_Name  Actor_ID                      Actor_Name Actor_TrigDate Actor_TermDate  Actor_Location  Alliance_ID Alliance_Start Alliance_End                      Alliance_Type Actor_Member_Start Actor_Member_End  Alliance_Active  Actor_Is_Member  Both_Conditions    Crisis_Location
       185      BERLIN WALL       265 German Democratic Republic(265)     1961-08-31     1961-08-13              32         3285     1955-05-14   1991-07-01 Defense;NonAggression;Consultation         1955-05-14       1990-10-03                1                1                1 Central Europe(32)
       475 KOREAN LAND MINE       732                South Korea(732)     2015-08-31     2015-08-25              11         3240     1953-10-01   2030-12-31 Defense;NonAggression;Consultation         1953-10-01       2030-12-31                1                1                1      East Asia(11)
       475 KOREAN LAND MINE       732          

In [11]:
# ── Add "Alliance_Members" to actor-level datasets ─────────────────────────
import pandas as pd
from pathlib import Path

# --------------------------------------------------------------------------
# 1. paths
# --------------------------------------------------------------------------
FULL_CSV = Path("actor_level_data/ICB_ATOP_actor_level_20250702.csv")
FILT_CSV = Path("actor_level_data/ICB_ATOP_actor_level_20250702_filtered.csv")
ATOP_MBR = Path(r"atop_5.1__.csv_\ATOP 5.1 (.csv)\atop5_1m.csv")
COW_CSV  = Path("COW-country-codes.csv")

# --------------------------------------------------------------------------
# 2. country-code → name mapping  (incl. three extra entities)
# --------------------------------------------------------------------------
cow = pd.read_csv(COW_CSV)[["CCode", "StateNme"]]
code2name = dict(zip(cow.CCode, cow.StateNme))
code2name.update({219: "Vichy France", 671: "Hejaz", 672: "Najd"})

# --------------------------------------------------------------------------
# 3. alliance-id → formatted member list  (Name(code);Name(code)…)
# --------------------------------------------------------------------------
roster = pd.read_csv(ATOP_MBR, usecols=["atopid", "member"])
roster["member"] = roster["member"].astype(int)

def fmt(lst):
    return ";".join(f"{code2name.get(c, 'Unknown')}({c})" for c in sorted(set(lst)))

alli2members = (roster.groupby("atopid")["member"]
                        .apply(fmt)
                        .to_dict())

# --------------------------------------------------------------------------
# 4. helper to append the column and resave
# --------------------------------------------------------------------------
def add_members_column(path: Path):
    df = pd.read_csv(path)
    df["Alliance_Members"] = df["Alliance_ID"].map(alli2members).fillna("")
    df.to_csv(path, index=False)
    print(f"✓ updated {path.name}  (rows: {len(df):,})")

# full file
add_members_column(FULL_CSV)

# regenerate filtered file from the updated full (Both_Conditions == 1)
full_df = pd.read_csv(FULL_CSV)
full_df[full_df["Both_Conditions"] == 1]\
       .to_csv(FILT_CSV, index=False)
print(f"✓ regenerated {FILT_CSV}")


✓ updated ICB_ATOP_actor_level_20250702.csv  (rows: 892,359)
✓ regenerated actor_level_data\ICB_ATOP_actor_level_20250702_filtered.csv


In [12]:
import pandas as pd
from pathlib import Path

FULL = Path("actor_level_data/ICB_ATOP_actor_level_20250702.csv")
df   = pd.read_csv(FULL)

# ------------------------------------------------------------------
# 1.  highest phase reached by each actor in each crisis
# ------------------------------------------------------------------
phase_by_actor = (df.groupby(["Crisis_ID", "Actor_ID"])["Crisis_Phase"]
                    .max()
                    .reset_index(name="Max_Phase"))

# ------------------------------------------------------------------
# 2.  average phases per crisis  (mean of actor-level maxima)
# ------------------------------------------------------------------
avg_phase_per_crisis = (phase_by_actor.groupby("Crisis_ID")["Max_Phase"]
                          .mean()
                          .reset_index(name="Avg_Phases"))

print("\nSample (first 10 crises):")
print(avg_phase_per_crisis.head(10).to_string(index=False))

# ------------------------------------------------------------------
# 3.  overall mean across crises
# ------------------------------------------------------------------
overall_mean = avg_phase_per_crisis["Avg_Phases"].mean()
print(f"\nOverall average number of phases per crisis: {overall_mean:.3f}")



Sample (first 10 crises):
 Crisis_ID  Avg_Phases
         1       789.0
         2       789.0
         3       789.0
         4       789.0
         5       789.0
         6      1052.0
         7       789.0
         8       789.0
         9       789.0
        10       789.0

Overall average number of phases per crisis: 803.177


In [13]:
# ── Rebuild Crisis_Phase using unique actor windows ───────────────
import pandas as pd
from pathlib import Path

FULL = Path("actor_level_data/ICB_ATOP_actor_level_20250702.csv")
FILT = Path("actor_level_data/ICB_ATOP_actor_level_20250702_filtered.csv")

# 1. load full actor-level data
df = pd.read_csv(FULL)

# 2. parse dates once for reliable ordering
df["Trig_dt"] = pd.to_datetime(df["Actor_TrigDate"], errors="coerce")
df["Term_dt"] = pd.to_datetime(df["Actor_TermDate"], errors="coerce")

# 3. build the unique-window table
windows = (df[["Crisis_ID","Actor_ID","Trig_dt","Term_dt"]]
           .drop_duplicates()
           .sort_values(["Crisis_ID","Actor_ID","Trig_dt","Term_dt"])
           .assign(Crisis_Phase=lambda x:
                   x.groupby(["Crisis_ID","Actor_ID"]).cumcount() + 1))

# 4. merge phase back to every alliance row
df = (df.drop(columns="Crisis_Phase", errors="ignore")      # if old column exists
        .merge(windows, on=["Crisis_ID","Actor_ID","Trig_dt","Term_dt"],
               how="left", validate="many_to_one"))

# 5. clean up helper columns
df = df.drop(columns=["Trig_dt","Term_dt"])

# 6. sanity check: new key must be unique
dupes = df.duplicated(["Crisis_ID","Actor_ID","Alliance_ID","Crisis_Phase"]).sum()
assert dupes == 0, f"Still {dupes} duplicates - inspect manually"
print("✓ Crisis_Phase rebuilt; duplicates with phase key = 0")

# 7. overwrite full file
df.to_csv(FULL, index=False)
print(f"✓ overwrote {FULL.name} (rows: {len(df):,})")

# 8. regenerate filtered subset (Both_Conditions == 1)
df_filt = df[df["Both_Conditions"] == 1].copy()
df_filt.to_csv(FILT, index=False)
print(f"✓ regenerated {FILT.name} (rows: {len(df_filt):,})")

# 9. quick stats
avg_phase = (df.groupby(["Crisis_ID","Actor_ID"])["Crisis_Phase"]
               .max()
               .groupby("Crisis_ID")
               .mean()
               .mean())
print(f"Overall average phases per crisis now: {avg_phase:.3f}")


✓ Crisis_Phase rebuilt; duplicates with phase key = 0
✓ overwrote ICB_ATOP_actor_level_20250702.csv (rows: 892,359)
✓ regenerated ICB_ATOP_actor_level_20250702_filtered.csv (rows: 5,102)
Overall average phases per crisis now: 1.018


In [ ]:
# ── 10. also save JSON versions ───────────────────────────────────
for path, data in [(FULL, df), (FILT, df_filt)]:
    js_path = path.with_suffix(".json")

    # stringify date columns for JSON compatibility
    date_cols = ["Crisis_Start", "Crisis_End",
                 "Alliance_Start", "Alliance_End",
                 "Actor_TrigDate", "Actor_TermDate"]
    data_out = data.copy()
    for col in date_cols:
        if col in data_out.columns:
            data_out[col] = data_out[col].astype(str)

    data_out.to_json(js_path, orient="records", indent=2)
    print(f"✓ wrote {js_path.name} (records: {len(data_out):,})")


In [14]:
# ── 10. also save JSON versions ───────────────────────────────────
for path, data in [(FULL, df), (FILT, df_filt)]:
    js_path = path.with_suffix(".json")

    # stringify date columns for JSON compatibility
    date_cols = ["Crisis_Start", "Crisis_End",
                 "Alliance_Start", "Alliance_End",
                 "Actor_TrigDate", "Actor_TermDate"]
    data_out = data.copy()
    for col in date_cols:
        if col in data_out.columns:
            data_out[col] = data_out[col].astype(str)

    data_out.to_json(js_path, orient="records", indent=2)
    print(f"✓ wrote {js_path.name} (records: {len(data_out):,})")


✓ wrote ICB_ATOP_actor_level_20250702.json (records: 892,359)
✓ wrote ICB_ATOP_actor_level_20250702_filtered.json (records: 5,102)
